In [1]:
import pandas as pd 
import numpy as np
from math import log10, floor

In [2]:
film = pd.read_csv('data/datasets/data_marvel.csv')

In [3]:
sales_2008 = pd.read_csv('data/datasets/sales_2008.csv')
sales_2009 = pd.read_csv('data/datasets/sales_2009.csv')
sales_2010 = pd.read_csv('data/datasets/sales_2010.csv')
sales_2011 = pd.read_csv('data/datasets/sales_2011.csv')
sales_2012 = pd.read_csv('data/datasets/sales_2012.csv')
sales_2013 = pd.read_csv('data/datasets/sales_2013.csv')
sales_2014 = pd.read_csv('data/datasets/sales_2014.csv')
sales_2015 = pd.read_csv('data/datasets/sales_2015.csv')
sales_2016 = pd.read_csv('data/datasets/sales_2016.csv')
sales_2017 = pd.read_csv('data/datasets/sales_2017.csv')
sales_2018 = pd.read_csv('data/datasets/sales_2018.csv')
sales_2019 = pd.read_csv('data/datasets/sales_2019.csv')
sales_2020 = pd.read_csv('data/datasets/sales_2020.csv')
sales_2021 = pd.read_csv('data/datasets/sales_2021.csv')

In [4]:
def generate_last_digits(index):
    num_digits = len(str(index))
    return index % (1 ** (num_digits - 5))

In [5]:
def format_number(value):
    if value >= 1e6:
        return f"{value / 1e6:.2f}M"
    elif value >= 1e3:
        return f"{value / 1e3:.2f}K"
    else:
        return f"{value:.2f}"

In [6]:
def comics_creator(data, date):
    data['Publish Date'] = date
    second_column = data.pop('Publish Date')
    data.insert(1, 'Publish Date', second_column)
    data['Price'] = data['Price'].replace('[\$,]', '', regex=True).astype(float)
    data['Est. units'] = data['Est. units'].replace(',', '', regex=True).astype(float)
    data['Total'] = data['Price'] * data['Est. units']
    data['Comic_ID'] = data.index
    
    data_list = []
    df = pd.DataFrame(columns=['Comic_ID', 'Publish Date', 'Comic-book Title', 'Price', 'Publisher', 'Est. units', 'Total', 'Unformatted Total'])
    
    for index, result in data.iterrows():
        local = {}
        local['Comic_ID'] = result['Comic_ID']
        local['Publish Date'] = result['Publish Date']
        local['Comic-book Title'] = result['Comic-book Title']
        local['Price'] = result['Price']
        local['Publisher'] = result['Publisher']
        local['Est. units'] = result['Est. units']
        local['Unformatted Total'] = result['Total']
        data_list.append(local)
    df = pd.DataFrame(data_list)
        
    df['Total'] = df['Unformatted Total'].apply(format_number)
        
    df['Publish Date'] = df['Publish Date'].astype(int)
    last_two_digits = str(date[-2:])
    identifier = {'legend': 315, 'date': last_two_digits, 'index': df.index}
    last_digits = generate_last_digits(df.index)
    df['Comic_ID'] = str(identifier['legend']) + str(identifier['date']) + last_digits.astype(str) + identifier['index'].astype(str)
    
    file_name = f'comic_{date}.csv'
    df.to_csv(f'data/comics_by_year/{file_name}', index=False)   
    return df

In [7]:
def box_characters_creator(df, check_list, date):
    matches = df[df['Comic-book Title'].str.contains('|'.join(check_list))]
    result_dict = {}
    for valor_b in check_list:
        matching_rows = matches[matches['Comic-book Title'].str.contains(valor_b)]
        if not matching_rows.empty:
            result_dict[valor_b] = {'Name': valor_b, 'Total': matching_rows['Unformatted Total'].sum()} 
    result_df = pd.DataFrame.from_dict(result_dict, orient='index').reset_index()
    result_df.columns = ['Character_Name', 'Character_ID', 'Unformatted Total']
    
    identifier = []
    check_letter = {'box': 21524}
    identifier.append(check_letter.get('box',0))
    result_df['Date'] = date
    result_df['Date'] = result_df['Date'].astype(int)
    last_two_digits = result_df['Date'] % 100
    identifier.append(last_two_digits.unique()[0])
    identifier.append(result_df.index)
    result_df['Character_ID'] = result_df.index
    result_df['Character_ID'] = identifier[0] * 10000 + identifier[1] * 100 + identifier[2]
    
    result_df['Total'] = result_df['Unformatted Total'].apply(format_number)
    
    result_df = result_df[['Character_ID', 'Date', 'Character_Name', 'Total', 'Unformatted Total']]
    file_name = f'box_office_characters_{date}.csv'
    result_df.to_csv(f'data/characters_by_year/{file_name}', index=False)
    return result_df

In [8]:
def film_creator(df):
    identifier = []
    check_letter = {'m': 13}
    identifier.append(check_letter.get('m',0))
    
    df['Opening weekend(North America)'] = df['Opening weekend(North America)'].replace('[\$,]', '', regex=True).astype(float)
    df['North America'] = df['North America'].replace('[\$,]', '', regex=True).astype(float)
    df['Other territories'] = df['Other territories'].replace('[\$,]', '', regex=True).astype(float)
    df['Worldwide'] = df['Worldwide'].replace('[\$,]', '', regex=True).astype(float)
    
    df['For Opening weekend(North America)'] = df['Opening weekend(North America)'].apply(format_number)
    df['For North America'] = df['North America'].apply(format_number)
    df['For Other territories'] = df['Other territories'].apply(format_number)
    df['For Worldwide'] = df['Worldwide'].apply(format_number)
    
    df['Release year'] = df['Release date(United States)'].str.extract(r'(\d{4})')
    df['Release year'] = df['Release year'].astype(int)
    df.drop('Release date(United States)', axis=1, inplace=True)
    second_column = df.pop('Release year')
    df.insert(2, 'Release year', second_column)
    last_two_digits = df['Release year'] % 100
    
    identifier.append(last_two_digits.unique()[0])
    identifier.append(df.index)
    
    df['Film_ID'] = df.index
    df['Film_ID'] = identifier[0] * 10000 + identifier[1] * 100 + identifier[2]
    
    cols = list(df.columns)
    cols = ['Film_ID'] + cols[:-1]
    df = df[cols]
    
    df.to_csv(f'data/box_office_film.csv', index=False)
    return df

In [9]:
def join_df(dfs, name):
    df = pd.concat(dfs, ignore_index=True)
    file_name = f'{name}.csv'
    df.to_csv(f'data/{file_name}', index=False)

In [10]:
def id_generator(df, check, id_name):
    identifier = []
    check_letter = {'box':21524,'ch': 38,'co':315, 'd': 4, 'h': 8, 'm': 13}
    identifier.append(check_letter.get(check, 0))
    identifier.append(df.index)
    df[id_name] = identifier[0] * 100 + identifier[1]
    cols = list(df.columns)
    cols = [id_name] + cols[0:]
    df = df[cols]
    return df

In [11]:
def save_csv(data, name):
    file_name = f'{name}.csv'
    data.to_csv(f'data/{file_name}', index=False)

In [12]:
def add_date(data):
    data['Release year'] = data['Release date(United States)'].str.extract(r'(\d{4})')
    data['Release year'] = data['Release year'].astype(int)
    data.drop('Release date(United States)', axis=1, inplace=True)
    second_column = data.pop('Release year')
    data.insert(2, 'Release year', second_column)
    return data

In [13]:
check_list = [
    'Absorbing Man', 'A.I.M.', 'Anole', 'Apocalypse', 'Archangel', 'Avalanche', 'Bastion', 'Beast',
    'Beyonder', 'Black Panther', 'Black Widow', 'Blob', 'Cable', 'Cannonball', 'Captain America',
    'Cassandra Nova', 'Chamber', 'Colossus', 'Crossbones', 'Cypher', 'Daredevil', 'Dark Phoenix',
    'Deadpool', 'Doctor Strange', 'Domino', 'Dust', 'Emma Frost', 'Exodus', 'Falcon', 'Fantomex',
    'Feral', 'Forge', 'Gambit', 'Gideon', 'Grandmaster', 'Grim Reaper', 'Havok', 'Hela', 'Hellion',
    'Hope Summers', 'Hulk', 'Iceman', 'Iron Man', 'Iron Patriot', 'Jean Grey', 'Jean Grey', 'Jubilee',
    'Juggernaut', 'Juggernaut', 'Kang', 'Karma', 'Korvac', 'Klaw', 'Lady Deathstrike', 'Living Laser',
    'Loki', 'Longshot', 'Luke Cage', 'M.O.D.O.K.', 'Magneto', 'Marrow', 'Mastermind', 'Master Mold',
    'Marauders', 'Marvel Boy', 'Multiple Man', 'Mystique', 'Nightcrawler', 'Omega Red', 'Onslaught',
    'Pixie', 'Polaris', 'Prodigy', 'Professor X', 'Psylocke', 'Red Skull', 'Rhino', 'Rockslide',
    'Rogue', 'Scarlet Witch', 'Sebastian Shaw', 'Selene', 'Shadowcat', 'Shadow King', 'Shatterstar',
    'Silver Samurai', 'Sinister Six', 'Skrulls', 'Sage', 'Spider-Man', 'Storm', 'Strong Guy', 'Sunspot',
    'Surge', 'Taskmaster', 'Thanos', 'The Hand', 'The Hood', 'Thor', 'U-Foes', 'Ultron', 'Vision',
    'Warpath', 'Wasp', 'William Stryker', 'Wolverine', 'Wolverine', 'X-23','X-Men', 'Avengers',
    'Ghost Rider', 'Venom', 'Dark Phoenix', 'Inhumans', 'Guardians of the Galaxy', 'Logan', 'Fantastic Four',
    'Punisher'
]




In [14]:
film = film_creator(film)

,Film_ID,Title,Distributor(s),Release year,Budget (millions),Opening weekend(North America),North America,Other territories,Worldwide,For Opening weekend(North America),For North America,For Other territories,For Worldwide
0,130800,Iron Man,Paramount Pictures,2008,$140,98618668.0,318412101.0,2.667621e+08,5.851742e+08,98.62M,318.41M,266.76M,585.17M
1,130801,The Incredible Hulk,Universal Pictures,2008,$150,55414050.0,134806913.0,1.286206e+08,2.634276e+08,55.41M,134.81M,128.62M,263.43M
2,130802,Punisher: War Zone,Lionsgate Films,2008,$35,4271451.0,8050977.0,2.049059e+06,1.010004e+07,4.27M,8.05M,2.05M,10.10M
3,130803,X-Men Origins: Wolverine,20th Century Fox,2009,$150,85058003.0,179883157.0,1.931797e+08,3.730629e+08,85.06M,179.88M,193.18M,373.06M
4,130804,Iron Man 2,Paramount Pictures,2010,$200,128122480.0,312433331.0,3.115000e+08,6.239333e+08,128.12M,312.43M,311.50M,623.93M
5,130805,Thor,Paramount Pictures,2011,$150,65723338.0,181030624.0,2.682960e+08,4.493266e+08,65.72M,181.03M,268.30M,449.33M
6,130806,X-Men: First Class,20th Century Fox,2011,$160,55101604.0,146408305.0,2.072158e+08,3.536241e+08,55.10M,146.41M,207.22M,353.62M
7,130807,Captain America: The First Avenger,Paramount Pictures,2011,$140,65058524.0,176654505.0,1.939153e+08,3.705698e+08,65.06M,176.65M,193.92M,370.57M
8,130808,Ghost Rider: Spirit of Vengeance,Sony Pictures,2012,$57,22115334.0,51774002.0,8.078993e+07,1.325639e+08,22.12M,51.77M,80.79M,132.56M
9,130809,The Avengers,Walt Disney Studios Motion Pictures,2012,$220,207438708.0,623357910.0,8.954551e+08,1.518813e+09,207.44M,623.36M,895.46M,1518.81M


In [15]:
df_08 = comics_creator(sales_2008,'2008')
df_09 = comics_creator(sales_2009, '2009')
df_10 = comics_creator(sales_2010, '2010')
df_11 = comics_creator(sales_2011, '2011')
df_12 = comics_creator(sales_2012, '2012')
df_13 = comics_creator(sales_2013, '2013')
df_14 = comics_creator(sales_2014, '2014')
df_15 = comics_creator(sales_2015, '2015')
df_16 = comics_creator(sales_2016, '2016')
df_17 = comics_creator(sales_2017, '2017')
df_18 = comics_creator(sales_2018, '2018')
df_19 = comics_creator(sales_2019, '2019')
df_20 = comics_creator(sales_2020, '2020')
df_21 = comics_creator(sales_2021, '2021')

In [16]:
comic = join_df([
    df_08, df_09, df_10,
    df_11, df_12, df_13,
    df_14, df_15, df_16,
    df_17, df_18, df_19, 
    df_20, df_21], 
    name = 'box_office_comics' 
    )

In [17]:
characters_08 = box_characters_creator(df_08, check_list, '2008')
characters_09 = box_characters_creator(df_09, check_list, '2009')
characters_10 = box_characters_creator(df_10, check_list, '2010')
characters_11 = box_characters_creator(df_11, check_list, '2011')
characters_12 = box_characters_creator(df_12, check_list, '2012')
characters_13 = box_characters_creator(df_13, check_list, '2013')
characters_14 = box_characters_creator(df_14, check_list, '2014')
characters_15 = box_characters_creator(df_15, check_list, '2015')
characters_16 = box_characters_creator(df_16, check_list, '2016')
characters_17 = box_characters_creator(df_17, check_list, '2017')
characters_18 = box_characters_creator(df_18, check_list, '2018')
characters_19 = box_characters_creator(df_19, check_list, '2019')
characters_20 = box_characters_creator(df_20, check_list, '2020')
characters_21 = box_characters_creator(df_21, check_list, '2021')


In [18]:
join_df(
        [characters_08, characters_09, characters_10, 
        characters_11, characters_12, characters_13,
        characters_14, characters_15, characters_16,
        characters_17, characters_18, characters_19,
        characters_20, characters_21], 
        name='box_office_characters'
        )

In [19]:
char = pd.read_csv('data/box_office_characters.csv')
unique = pd.DataFrame(char['Character_Name'].unique())
unique['Name'] = unique[0]
unique.drop(0, axis=1, inplace=True)

In [20]:
id_generator(unique, 'ch', 'Char_id')
save_csv(unique, 'characters')
unique

,Name,Char_id
0,Beast,3800
1,Black Panther,3801
2,Cable,3802
3,Captain America,3803
4,Colossus,3804
5,Daredevil,3805
6,Deadpool,3806
7,Hulk,3807
8,Iron Man,3808
9,Jean Grey,3809
